# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

План проекта:
- 1 Загрузка и подготовка данных
- - 1.1 Обработка дубликатов
- 2 Обучение модели
- - 2.1 Разделение данных на выборки
- - 2.2 Обучение модели
- 3 Подготовка к расчёту прибыли
- - 3.1 Рассчет достаточного объёма сырья для безубыточной разработки новой скважины
- - 3.2 Функция расчета по регионам
- 4 Расчет прибыли и рисков
- - 4.1 Функция расчета прибыли
- - 4.2 Техника Bootstrap
- 5 Общий вывод

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

###Модели###
from sklearn.linear_model import LinearRegression

###Метрики###
from sklearn.metrics import mean_squared_error
from scipy import stats as st

df1 = pd.read_csv('/datasets/geo_data_0.csv')
df2 = pd.read_csv('/datasets/geo_data_1.csv')
df3 = pd.read_csv('/datasets/geo_data_2.csv')

df1.info()
df2.info()
df3.info()
df1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


### Вывод 
Пропусков нет, значение столбцы f0,f1,f2 нам не дано,  столбец 'products' содержит в себе целевой признак Попробуем далее проверить 3 датасета на наличие явных и неявных дубликатов.

### Обработка дубликатов

In [2]:
dfs = [df1, df2, df3]

print('Дубликаты по номеру скважины локации 1:', len(df1['id'])- len(df1['id'].drop_duplicates()))

print('Дубликаты по номеру скважины локации 2:', len(df2['id'])- len(df2['id'].drop_duplicates()))

print('Дубликаты по номеру скважины локации 3:', len(df3['id'])- len(df3['id'].drop_duplicates()))

ids = df1["id"]
print(df1[ids.isin(ids[ids.duplicated()])].sort_values("id"))

for df in dfs:
    df.drop_duplicates(subset='id', keep = False, inplace = True, ignore_index=True)    

Дубликаты по номеру скважины локации 1: 10
Дубликаты по номеру скважины локации 2: 4
Дубликаты по номеру скважины локации 3: 4
          id        f0        f1         f2     product
66136  74z30  1.084962 -0.312358   6.990771  127.643327
64022  74z30  0.741456  0.459229   5.153109  140.771492
51970  A5aEY -0.180335  0.935548  -2.094773   33.020205
3389   A5aEY -0.039949  0.156872   0.209861   89.249364
69163  AGS9W -0.933795  0.116194  -3.655896   19.230453
42529  AGS9W  1.454747 -0.479651   0.683380  126.370504
931    HZww2  0.755284  0.368511   1.863211   30.681774
7530   HZww2  1.061194 -0.373969  10.430210  158.828695
63593  QcMuo  0.635635 -0.473422   0.862670   64.578675
1949   QcMuo  0.506563 -0.323775  -2.215583   75.496502
75715  Tdehs  0.112079  0.430296   3.218993   60.964018
21426  Tdehs  0.829407  0.298807  -0.049563   96.035308
92341  TtcGQ  0.110711  1.022689   0.911381  101.318008
60140  TtcGQ  0.569276 -0.104876   6.440215   85.350186
89582  bsk9y  0.398908 -0.400253 

#### Вывод
Мы видим, что явных дубикатов нет. Однако есть одинаковые уникальные номера скважин со абсолютно ранзыми признаками. Мы могли бы вычслить среднее из наблюдений по одному номеру, но из ТЗ нам не даны пояснения, за что отвечают признаки f0, f1, f2 поэтому для чистоты данных мы удалим все наблюдения по данным уникальным номерам скважин (Мы не знаем, какие наблюдения корректны)

## Обучение и проверка модели

### Разделение данных на выборки

In [3]:
features_1 = df1.drop(['id', 'product'], axis = 1)
target_1 = df1['product']

features_2 = df2.drop(['id', 'product'], axis = 1)
target_2 = df2['product']

features_3 = df3.drop(['id', 'product'], axis = 1)
target_3 = df3['product']

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, 
                                                                              test_size=0.25, random_state=12345)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2,
                                                                              test_size=0.25, random_state=12345)
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(features_3, target_3,
                                                                              test_size=0.25, random_state=12345)
features_train_list = [features_train_1, features_train_2, features_train_3]
features_valid_list = [features_valid_1, features_valid_2, features_valid_3]
target_train_list = [target_train_1, target_train_2, target_train_3]
target_valid_list = [target_valid_1, target_valid_2, target_valid_3]


### Обучение модели

In [4]:
model = LinearRegression()
d = {}
for i in range(len(dfs)):
    model.fit(features_train_list[i], target_train_list[i])
    d["predictions_valid_" + str(i)] = model.predict(features_valid_list[i])
    rmse = mean_squared_error(target_valid_list[i], d["predictions_valid_" + str(i)])**0.5
    mean = d["predictions_valid_" + str(i)].mean()
    print('----------------------------------------------------------------------------------------')
    print(f'В регионе №{i+1} средний запас предсказанного сырья составил:{mean}, а RMSE модели:{rmse}')

----------------------------------------------------------------------------------------
В регионе №1 средний запас предсказанного сырья составил:92.42384109947359, а RMSE модели:37.716904960382735
----------------------------------------------------------------------------------------
В регионе №2 средний запас предсказанного сырья составил:68.98311857983123, а RMSE модели:0.8914901390348537
----------------------------------------------------------------------------------------
В регионе №3 средний запас предсказанного сырья составил:95.11622302076478, а RMSE модели:39.975543264382345


#### Вывод
Мы видим, как по результатам предсказанной доли сырья на равне лидируют регионы №1 и 3, в то время как регион номер 2 отстает. В то же время мы видим, значительную разницу в точности предсказаний по регионам и по данному параметру регион № 2 является самым менее рисковым и предсказуемым.

## Подготовка к расчёту прибыли

### Рассчет достаточного объёма сырья для безубыточной разработки новой скважины

In [5]:
budget = 10_000_000_000 #бюджет на регион
number_of_wells = 200 #кол-во скважин в регионе
product_income = 450_000 #доход с 1 тысячи баррелей
one_well_income = budget/number_of_wells #минимальный обьём прибыли по каждой скважине
one_well_barrels = one_well_income/one_well_income #минимальный обьём пр-ва по каждой скважине


one_well_income = budget/number_of_wells
one_well_barrels = one_well_income/product_income


region_one_mean = df1['product'].mean()
region_two_mean = df2['product'].mean()
region_three_mean = df3['product'].mean()

minimum_profit = budget/product_income
minimum_profit_1_well = minimum_profit/number_of_wells
print(f'Минимальный обьем добычи для окупания затрат для 1 скважины составляет:{minimum_profit_1_well:.4}')
print(f'Минимальный обьем добычи для окупания затрат для региона:{minimum_profit:.6}')


Минимальный обьем добычи для окупания затрат для 1 скважины составляет:111.1
Минимальный обьем добычи для окупания затрат для региона:22222.2


#### Вывод
Итого минимальный обьем для окупания затрат составляет 111 тысяч баррелей, а для окупания затрат всего региона - 22222 тысячи баррелей

### Функция расчета по регионам

In [6]:
def profit_area(area):
    area = pd.Series(area)
    area = area.sort_values(ascending=False)
    area = area[0:200]
    area_top200_mean = area.mean()
    print(f'- Разница между минимальным обьемом добычи для прибыли и топ 200 предсказанных скважин по региону: {area_top200_mean - minimum_profit_1_well}')
    
for i in range(len(dfs)):
    print('Данные по региону',i+1)
    profit_area(d["predictions_valid_" + str(i)])
    if i+1 == 1:
        print(f'- Разница между минимальным обьемом добычи для прибыли и среднем по региону: {region_one_mean - minimum_profit_1_well}')
    if i+1 == 2:
        print(f'- Разница между минимальным обьемом добычи для прибыли и среднем по региону: {region_two_mean - minimum_profit_1_well}')
    if i+1 == 3:
        print(f'- Разница между минимальным обьемом добычи для прибыли и среднем по региону: {region_three_mean - minimum_profit_1_well}')
    print('-----------------------------------------------')
    


                     

    

Данные по региону 1
- Разница между минимальным обьемом добычи для прибыли и топ 200 предсказанных скважин по региону: 42.91061854068016
- Разница между минимальным обьемом добычи для прибыли и среднем по региону: -18.611945132176672
-----------------------------------------------
Данные по региону 2
- Разница между минимальным обьемом добычи для прибыли и топ 200 предсказанных скважин по региону: 27.612214537959943
- Разница между минимальным обьемом добычи для прибыли и среднем по региону: -42.28696338445938
-----------------------------------------------
Данные по региону 3
- Разница между минимальным обьемом добычи для прибыли и топ 200 предсказанных скважин по региону: 38.83378624154918
- Разница между минимальным обьемом добычи для прибыли и среднем по региону: -16.11234424343033
-----------------------------------------------


#### Вывод
Мы видим, что если брать просто среднее по добычи по региону, то все регионы имеют отрицательную прибыль. Это неудивительно, т.к. мы брали среднее по всем скважинам в регионе, однако показатель с самой низкой отрицательной прибылью имеет регион №3. Попробуем взять топ 200 скважин по предсказаниям в каждом регоине и посчитать среднее у них. Здесь уже все регионы имеют положительную прибыль, а лидирует по этим показателям регион №1. Попробуем посчитать прибыль и найти доверительный интервал

## Расчёт прибыли и рисков 

### Функция расчета прибыли

In [7]:
predictions_one = pd.Series(d["predictions_valid_" + str(0)])
predictions_one = predictions_one.reset_index(drop=True)
target_one = target_valid_list[0].reset_index(drop=True)

predictions_two = pd.Series(d["predictions_valid_" + str(1)])
target_two = target_valid_list[1].reset_index(drop=True)

predictions_three = pd.Series(d["predictions_valid_" + str(2)])
target_three = target_valid_list[2].reset_index(drop=True)
#Для удобства сохраним все в отдельные переменные и обновим индексацию для дальнейших расчетов


In [8]:
def profit(target, predictions, count):
    profit=0
    probs_sorted = predictions.sort_values(ascending=False).head(200)
    selected = target[probs_sorted.index][:count]    
    for well in selected:
        profit+=(well-one_well_barrels)*product_income
    return profit

profit_1 = profit(predictions_one, target_one,200)
print(f'Для 1 региона, прибыль по 200 лучшим скважинам из предсказанных, составляет {profit_1/1000000:.6} млн.')

profit_2 = profit(predictions_two, target_two,200)
print(f'Для 2 региона, прибыль по 200 лучшим скважинам из предсказанных, составляет {profit_2/1000000:.6} млн.')

profit_3 = profit(predictions_three, target_three,200)
print(f'Для 3 региона, прибыль по 200 лучшим скважинам из предсказанных, составляет {profit_3/1000000:.6} млн.')


Для 1 региона, прибыль по 200 лучшим скважинам из предсказанных, составляет 541.62 млн.
Для 2 региона, прибыль по 200 лучшим скважинам из предсказанных, составляет 2366.41 млн.
Для 3 региона, прибыль по 200 лучшим скважинам из предсказанных, составляет -306.31 млн.


#### Вывод
- Немного изменили функцию, написанную выше, теперь уже для расчет прибыли по предсказаниям для 200 лучших скважин, которая предсказала модель
- Получили прогнозируемые прибыли для каждого региона. Из них следует, что потенциально самым прибыльным является регион №2, далее посчитаем риски и найдем доверительный интервал для каждого региона

### Техника Bootstrap

In [9]:
def inteval (target, predictions):
    state = np.random.RandomState(12345)
    values = []
    counter=0
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        preds_subsample = predictions[target_subsample.index]
        
        values.append(profit(target_subsample, preds_subsample, 500))
        
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    
    print('Вероятность получения отрицательной прибыли:',st.percentileofscore(values, 0),'%')
    print('Средняя прибыль:',values.mean())
    print('Нижняя граница 95%-го квантиля:', lower)
    print('Верхняя граница 95%-го квантиля:', upper)
    print('-------------------------------------------')

print('Данные по региону 1')
inteval(target_one, predictions_one)
print('Данные по региону 2')
inteval(target_two, predictions_two)
print('Данные по региону 3')
inteval(target_three, predictions_three)        

Данные по региону 1
Вероятность получения отрицательной прибыли: 5.5 %
Средняя прибыль: 439121098.4851674
Нижняя граница 95%-го квантиля: -122935263.44364947
Верхняя граница 95%-го квантиля: 962698056.251455
-------------------------------------------
Данные по региону 2
Вероятность получения отрицательной прибыли: 2.2 %
Средняя прибыль: 486100778.23722804
Нижняя граница 95%-го квантиля: 54885418.264950104
Верхняя граница 95%-го квантиля: 906983342.359008
-------------------------------------------
Данные по региону 3
Вероятность получения отрицательной прибыли: 13.0 %
Средняя прибыль: 330136466.74162376
Нижняя граница 95%-го квантиля: -202802559.09418762
Верхняя граница 95%-го квантиля: 868527323.4806209
-------------------------------------------


#### Вывод
Как мы видим из данных выше - регион 2 является самым перспективным для последующих работ. Регион сочетает в себе:
- Наилучшую прогнозируемость (RMSE модели составил всего ~0.89)
- Наибольшую среднюю прыбыль по предсказанным скважинам ~ 439121098
- Наименьший риск попадания значения прибыли в отрицательные значения: ~2.2%
Именно её мы и рекомендуем для бурения скважин

## Общий вывод
Мы взяли данные по 3 регионам, выявили дубликаты по уникальному номеру скважины, разбили данные на 2 выборки: валидационную и обучающую. Посчитали минимальный обьем для добычи по регионам для получения положительной прибыли и сравнили эти значения с уже имеющимся данными. Далее посчитали фактическую добычу по лучшим пресказаниям модели по каждому региону и написали функцию для подсчета прибыли по регионам. Далее техникой bootstrap нашли доверительный интервал 95%-го квантиля, шанс того, что итоговое значение прибыли попадет в отрицательные значения и среднюю прибыль по региону. По всем этим параметрам оказался лучшим регион номер 2, его мы и рекомендуем для дальнеших работ.